In [38]:
class Edge(object):
    def __init__(self, u, v, w):
        self.source = u
        self.target = v
        self.capacity = w

    def __repr__(self):
        return "%s->%s:%s" % (self.source, self.target, self.capacity)


class FlowNetwork(object):
    def  __init__(self):
        self.adj = {}
        self.flow = {}

    def AddVertex(self, vertex):
        self.adj[vertex] = []

    def GetEdges(self, v):
        return self.adj[v]

    def AddEdge(self, u, v, w = 0):
        if u == v:
            raise ValueError("u == v")
        edge = Edge(u, v, w)
        redge = Edge(v, u, 0)
        edge.redge = redge
        redge.redge = edge
        self.adj[u].append(edge)
        self.adj[v].append(redge)
        # Intialize all flows to zero
        self.flow[edge] = 0
        self.flow[redge] = 0

    def FindPath(self, source, target, path):
        result=None
        if source == target:
            return path
        for edge in self.GetEdges(source):
            residual = edge.capacity - self.flow[edge]
            if residual > 0 and not (edge, residual) in path:
                result = self.FindPath(edge.target, target, path + [(edge, residual)])
            if result != None:
                return result

    def MaxFlow(self, source, target):
        path = self.FindPath(source, target, [])
        #print 'path after enter MaxFlow: %s' % path
        #for key in self.flow:
        #      print '%s:%s' % (key,self.flow[key])
        #print '-' * 20
        while path != None:
            flow = min(res for edge, res in path)
            for edge, res in path:
                self.flow[edge] += flow
                self.flow[edge.redge] -= flow
            #for key in self.flow:
            #    print '%s:%s' % (key,self.flow[key])
            path = self.FindPath(source, target, [])
            #print 'path inside of while loop: %s' % path
            #for key in self.flow:
            #    print '%s:%s' % (key,self.flow[key])
        return sum(self.flow[edge] for edge in self.GetEdges(source))

In [39]:
import itertools as it
gr = FlowNetwork()
fourcomb=[]
threecomb=[]
cards=[i for i in range(1,28)]
for i in it.combinations(cards,4):
    fourcomb.append(i)
for i in it.combinations(cards,3):
    for j in it.permutations(i,3):
        threecomb.append(j)
map(gr.AddVertex,fourcomb)
map(gr.AddVertex,threecomb)
for i in fourcomb:
    for j in it.permutations(i,3):
        gr.AddEdge(i,j,1)

In [40]:
gr.AddVertex('root')
gr.AddVertex('end')
for i in fourcomb:
    gr.AddEdge('root',i,1)
for i in threecomb:
    gr.AddEdge(i,'end',1)

In [41]:
print gr.MaxFlow('root','end')

RuntimeError: maximum recursion depth exceeded in cmp